In [1]:
# 전이학습
# 기학습된 Pretrained Network VGG16을 이용

from tensorflow.keras.applications import VGG16

In [2]:
model_base = VGG16(weights='imagenet',
                   include_top=False,
                   input_shape=(150, 150, 3)) #150by150에 COLOR니까 3
print(model_base.summary())

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [3]:
import numpy as np
import tensorflow as tf   #tensorflow 기본
from tensorflow.keras.models import Sequential   #모델 box 정의
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Flatten, Dense   #모델 BOX Input, Output layer 정의
from tensorflow.keras.optimizers import Adam    #알고리즘 담당
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
# ImageDataGenerator 객체 생성 (rescale로 정규화)
Train_Datagen = ImageDataGenerator(rescale=1/255)
Test_Datagen = ImageDataGenerator(rescale=1/255)

In [5]:
train_dir = "./cats_and_dogs_small/train" 
test_dir = "./cats_and_dogs_small/test"

In [6]:
# ImageDataGenerator 설정
train_generator = Train_Datagen.flow_from_directory(
    train_dir,    # 학습용 이미지를 가져올 폴더
    classes=['cats', 'dogs'], # cats 폴더의 이미지 label을 0으로, 
                              # dogs 폴더의 이미지는 label을 1로 설정
    target_size=(150, 150),   # image를 resize
    batch_size =20,           # 한 번에 20개의 이미지만 가져와서 학습
    class_mode='binary'       # 이진분류인 경우 설정 (개, 고양이 둘 중에 하나)
)

test_generator = Test_Datagen.flow_from_directory(
    test_dir,    # 학습용 이미지를 가져올 폴더
    classes=['cats', 'dogs'], # cats 폴더의 이미지 label을 0으로, 
                              # dogs 폴더의 이미지는 label을 1로 설정
    target_size=(150, 150),   # image를 resize
    batch_size =10,           # 한 번에 10개의 이미지만 가져와서 학습
    class_mode='binary'       # 이진분류인 경우 설정 (개, 고양이 둘 중에 하나)
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [7]:
# CNN Model 구현
model = Sequential()
model_base = VGG16(weights='imagenet',
                   include_top=False,
                   input_shape=(150, 150, 3)) #150by150에 칼라니까 3
model.add(model_base)

In [8]:
# Classification 하는 부문은 구현해야 함.
# FC layer(DNN)의 input Layer
model.add(Flatten())   #전체 4차원에서 2차원으로 바꿔주는 것
# output layer
model.add(Dense(units=1, activation='sigmoid')) # 고양이인지 개인지 0또는 1로 판단

In [9]:
# model이 어떻게 동작하는지를 지정
model.compile(optimizer=Adam(learning_rate=0.0001),
             loss='binary_crossentropy',
             metrics=['accuracy'])

In [ ]:
# 설정이 다 되었으면 모델을 학습
model.fit(train_generator,
         steps_per_epoch=100,
         epochs=30,
         verbose=1,
         validation_data=test_generator,
          validation_steps=50)

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 1)                 8193      
                                                                 
Total params: 14,722,881
Trainable params: 14,722,881
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# predict, 평가해서 학습이 잘 되었는지 안 되었는지 항상 확인해야 함.